In [ ]:
!pip install birdy
!pip install ratelimiter

In [ ]:
import json, os, sys, time
from zipfile import ZipFile
from birdy.twitter import AppClient, UserClient, TwitterRateLimitError
from ratelimiter import RateLimiter

CONSUMER_KEY = 'gzufdJHhRm8uep91nBr9yJGwX'
CONSUMER_SECRET = 'qMx31ZTe6qmwRrvXeRldE04V8pKeHuBLYnmm991Mk4tOYwdyCq'

OUTPUT_DIR = '/content/drive/MyDrive/APRD6343/Network Analysis'
MAX_TWEETS = 10000
max_id = None
_client = None

def client(consumer_key=None, consumer_secret=None):
  global _client
  if consumer_key is None:
    consumer_key = CONSUMER_KEY
  if consumer_secret is None:
    consumer_secret = CONSUMER_SECRET
  if _client is None:
    _client = AppClient(consumer_key, consumer_secret)
    access_token = _client.get_access_token()
    _client = AppClient(consumer_key, consumer_secret, access_token)
  return _client

def limited(until):
  duration = int(round(until - time.time()))
  print('Rate limited, sleeping for {:d} seconds'.format(duration))

@RateLimiter(max_calls=440, period=60*15, callback=limited)
def fetch_tweets(query, consumer_key=None, consumer_secret=None):
  global max_id
  print(f'Fetching: "{query}" TO MAX ID: {max_id}')
  try:
      tweets = client(consumer_key, consumer_secret).api.search.tweets.get(
          q=query,
          count=100,
          max_id=max_id).data['statuses']
  except TwitterRateLimitError:
    sys.exit("You've reach your Twitter API rate limit. Wait 15 minutes before trying again")
  try:
    id_ = min([tweet['id'] for tweet in tweets])
  except ValueError:
    return None
  if max_id is None or id_ <= max_id:
    max_id = id_ - 1
  return tweets

def initialize_max_id(file_list):
  global max_id
  for fn in file_list:
    n = int(fn.split('.')[0])
    if max_id is None or n < max_id:
      max_id = n - 1
  if max_id is not None:
    print('Found previously fetched tweets. Setting max_id to %d' % max_id)

def halt(_id):
  print('Reached historically fetched ID: %d' % _id)
  print('In order to re-fetch older tweets, remove tweets from ther output directory or output zip file.')
  sys.exit('\n!!IMPORTANT: Tweets older than 7 days will not be re-fetched')

def search_twitter(query, consumer_key=None, consumer_secret=None, newtweets=False, dozip=True, verbose=False):
  output_dir = os.path.join(OUTPUT_DIR, '_'.join(query.split()))
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  if dozip:
    fn = os.path.join(output_dir, '%s.zip' % '_'.join(query.split()))
    outzip = ZipFile(fn, 'a')
  if not newtweets:
    if dozip:
      file_list = [f for f in outzip.namelist() if f.endswith('.json')]
    else:
      file_list = [f for f in os.listdir(output_dir) if f.endswith('.json')]
    initialize_max_id(file_list)
  while True:
    try: 
      tweets = fetch_tweets(
          query,
          consumer_key=consumer_key,
          consumer_secret=consumer_secret)
      if tweets is None:
        print('Search Completed')
        if dozip:
          outzip.close()
        break
      for tweet in tweets: 
        if verbose:
          print(tweet['id'])
        fn = '%d.json' % tweet['id']
        if dozip:
          if fn in (file_list):
            outzip.close()
            halt(tweet['id'])
          else:
            outzip.writestr(fn, json.dumps(tweet, indent=4))
            file_list.append(fn)
        else:
          path = os.path.join(output_dir, fn)
          if fn in (file_list):
            halt(tweet['id'])
          else:
            with open(path, 'w') as outfile:
              json.dump(tweet, outfile, indent=4)
            file_list.append(fn)
        if len(file_list) >= MAX_TWEETS:
          if fn in (file_list):
            outzip.close()
    except:
      if dozip:
        outzip.close()
      raise

  

In [ ]:
#search_twitter('playstation 5 OR ps5')

In [ ]:
#search_twitter("xbox series x OR xbox series s")

In [ ]:
#search_twitter("nintendoswitch OR nintendo switch")

#Mention Networks

In [ ]:
import glob
import os
import shutil
import json
import csv 
import zipfile

In [ ]:
WORKING_DIR = '/content/drive/MyDrive/APRD6343/Network Analysis'

In [ ]:
tweetzipfiles = glob.glob('%s/tweets/*.zip' % WORKING_DIR)

In [ ]:
tweetzipfiles

['/content/drive/MyDrive/APRD6343/Network Analysis/tweets/playstation_5_OR_ps5.zip',
 '/content/drive/MyDrive/APRD6343/Network Analysis/tweets/nintendoswitch_OR_nintendo_switch.zip',
 '/content/drive/MyDrive/APRD6343/Network Analysis/tweets/xbox_series_x_OR_xbox_series_s.zip']

In [ ]:
# Do more segmenting

uniqueusers = {}

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    userwhotweeted = tweetjson['user']['screen_name']
    if userwhotweeted in uniqueusers:
      uniqueusers[userwhotweeted] += 1
    if userwhotweeted not in uniqueusers:
      uniqueusers[userwhotweeted] = 1

In [ ]:
len(uniqueusers)

21462

In [ ]:
userstoinclude = set()

usercount = 0
for auser in uniqueusers:
  if uniqueusers[auser] > 2:
    usercount += 1
    userstoinclude.add(auser)

print(len(userstoinclude))

1487


In [ ]:
edgelist = open('%s/console.edgelist.for.gephi.csv' % WORKING_DIR, 'w')
csvwriter = csv.writer(edgelist)
header = ['Source', 'Target']
csvwriter.writerow(header)

15

In [ ]:
print('Writing edge list')
count = 0

for tweetzipfile in tweetzipfiles:
  count += 1
  if count % 1000 == 0:
    print(count)
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    userwhotweeted = tweetjson['user']['screen_name']
    if userwhotweeted in userstoinclude:
      users = tweetjson['entities']['user_mentions']
      if len(users) > 0:
        for auser in users:
          screenname = auser['screen_name']
          row = [userwhotweeted, screenname]
          csvwriter.writerow(row)
edgelist.close()

Writing edge list


#Semantic Networks

In [ ]:
import nltk
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()
import glob
import os
import re
import shutil
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import csv
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import string
import itertools
import zipfile
import json
punctuation = string.punctuation
stopwordsset = set(stopwords.words("english"))
stopwordsset.add('rt')
stopwordsset.add("'s")

stopwordsset.add("amp")
stopwordsset.add("ve")
stopwordsset.add('•')
emojis = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
nltk.download('words')
words = set(nltk.corpus.words.words())
nltk.download('names')
stopwordsset.add(w.lower() for w in nltk.corpus.names.words())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


In [ ]:
#Removing urls
def removeURL(text):
  result = re.sub(r"http\S+", "", text)
  return result

#Extracting contextual words from a sentence
# tokenizing is taking out all the words in a sentence and turning it into tokens/words
def tokenize(text):
  #lower case
  text = text.lower()
  #split into individual words
  words = word_tokenize(text)
  return words

#stem - peaches : peach : reduce the number of repeated words
def stem(tokenizedtext):
  rootwords = []
  for aword in tokenizedtext:
    aword = ps.stem(aword)
    rootwords.append(aword)
  return rootwords

#removes useless words such as a, an, the
def stopWords(tokenizedtext):
  goodwords = []
  for aword in tokenizedtext:
    if aword not in stopwordsset:
      goodwords.append(aword)
  return goodwords

# feature reduction. taking words and getting their roots and graphing only the root words
def lemmatizer(tokenizedtext):
  lemmawords = []
  for aword in tokenizedtext:
    aword = wn.lemmatize(aword)
    lemmawords.append(aword)
  return lemmawords

#inputs a list of tokens and returns a list of unpunctuated tokens/words
def removePunctuation(tokenizedtext):
  nopunctwords = []
  for aword in tokenizedtext:
    if aword not in punctuation:
      nopunctwords.append(aword)
  cleanedwords = []
  for aword in nopunctwords:
    aword = aword.translate(str.maketrans('', '', string.punctuation))
    cleanedwords.append(aword)
  return cleanedwords

def deEmojify(tokenizedtext):
    deemojifiedwords = []
    for aword in tokenizedtext:
      aword = emojis.sub(r'',aword)
      deemojifiedwords.append(aword)
    return deemojifiedwords

def removeForeign(tokenizedtext):
  english = []
  for aword in tokenizedtext:
    res = [idx for idx in aword if not 
           re.findall("[^\u0000-\u05C0\u2100-\u214F\0-9]+", idx)]
    res = ''.join(res)
    if res in words or not res.isalpha():
      if len(res) > 2:
        english.append(res)
  return english

In [ ]:
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
from nltk.corpus import twitter_samples

def lemmatize_sentence(tokens):
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(wn.lemmatize(word, pos))
    return lemmatized_sentence

def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []
    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        token = wn.lemmatize(token, pos)
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stopwordsset))
for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stopwordsset))

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]
dataset = positive_dataset + negative_dataset
random.shuffle(dataset)
train_data = dataset[:7000]
test_data = dataset[7000:]

from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)
print("Accuracy is:", classify.accuracy(classifier, test_data))
print(classifier.show_most_informative_features(10))

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Accuracy is: 0.9973333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2083.4 : 1.0
                      :) = True           Positi : Negati =   1641.7 : 1.0
                     sad = True           Negati : Positi =     24.2 : 1.0
                     bam = True           Positi : Negati =     22.7 : 1.0
                follower = True           Positi : Negati =     20.3 : 1.0
                     x15 = True           Negati : Positi =     17.2 : 1.0
               community = True           Positi : Negati =     16.1 : 1.0
                 welcome = True           Positi : Negati =     15.6 : 1.0
                  arriv

In [ ]:
from time import sleep 

uniquewords = {}
uniqueposwords = {}
uniquenegwords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
        print(count)
    
    
    text = tweetjson['text']
    #natural language pre processing : clean the tweet 
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    deemojifiedtext = deEmojify(nopuncttext)
    englishtext = removeForeign(deemojifiedtext)
    sentiment = classifier.classify(dict([token, True] for token in englishtext))
    
    #print(tokenizedtext)
    #print(nostopwordstext)
    #print(lemmatizedtext)
    #print(nopuncttext)
    #print(deemojifiedtext)
    #print(englishtext)
    #print(sentiment)
    #sleep(2)

    for aword in englishtext:
        if aword in uniquewords:
            uniquewords[aword] += 1
        if aword not in uniquewords:
            uniquewords[aword] = 1

    if sentiment == 'Positive':
      for aword in englishtext:
        if aword in uniqueposwords:
            uniqueposwords[aword] += 1
        if aword not in uniqueposwords:
            uniqueposwords[aword] = 1

    if sentiment == 'Negative':
      for aword in englishtext:
        if aword in uniquenegwords:
            uniquenegwords[aword] += 1
        if aword not in uniquenegwords:
            uniquenegwords[aword] = 1

128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000


In [ ]:
print(len(uniquewords))
print(len(uniqueposwords))
print(len(uniquenegwords))

6842
4787
4115


In [ ]:
#  list of unique words: key - word; value = number of times the word appears.
wordstoinclude = set()
wordcount = 0
for aword in uniquewords:
    if uniquewords[aword] > 18: 
        wordcount += 1
        wordstoinclude.add(aword)

poswordstoinclude = set()
poswordcount = 0
for aword in uniqueposwords:
    if uniqueposwords[aword] > 9: 
        poswordcount += 1
        poswordstoinclude.add(aword)

negwordstoinclude = set()
negwordcount = 0
for aword in uniquenegwords:
    if uniquenegwords[aword] > 6: 
        negwordcount += 1
        negwordstoinclude.add(aword)

In [ ]:
print(wordcount)
print(poswordcount)
print(negwordcount)

930
914
933


In [ ]:
edgelist = open('%s/console.semantic.edgelist.for.gephi.csv' % WORKING_DIR,'w')
csvwriter = csv.writer(edgelist)
poslist = open('%s/pos.semantic.edgelist.for.gephi.csv' % WORKING_DIR,'w')
poswriter = csv.writer(poslist)
neglist = open('%s/neg.semantic.edgelist.for.gephi.csv' % WORKING_DIR,'w')
negwriter = csv.writer(neglist)

header = ['Source', 'Target', 'Type'] # add Type to mention this is undirected. in previous file, it was directed so didnt use it
csvwriter.writerow(header)
poswriter.writerow(header)
negwriter.writerow(header)

print('Writing Edge List')

uniquewords = {}
uniqueposwords = {}
uniquenegwords = {}
count = 0

for tweetzipfile in tweetzipfiles:
  zf = zipfile.ZipFile(tweetzipfile)
  for i, obj in enumerate(zf.infolist()):
    tweetjson = json.load(zf.open(obj))
    count += 1
    if count % 1000 == 0:
        print(count)
    
    text = tweetjson['text']
    #natural language pre processing : clean the tweet 
    nourlstext = removeURL(text)
    tokenizedtext = tokenize(nourlstext)
    nostopwordstext = stopWords(tokenizedtext)
    lemmatizedtext = lemmatizer(nostopwordstext)
    nopuncttext = removePunctuation(lemmatizedtext)
    deemojifiedtext = deEmojify(nopuncttext)
    englishtext = removeForeign(deemojifiedtext)
    sentiment = classifier.classify(dict([token, True] for token in englishtext))

    goodwords = []
    for aword in englishtext:
        if aword in wordstoinclude:
            goodwords.append(aword.replace(',', ''))
            
    allcombos = itertools.combinations(goodwords, 2) # set this to the number of words you want in each combination
    for acombo in allcombos:
        row = []
        for anode in acombo:
            row.append(anode)
        row.append('Undirected')
        csvwriter.writerow(row)

    if sentiment == 'Positive':
      poswords = []
      for aword in englishtext:
          if aword in poswordstoinclude:
              poswords.append(aword.replace(',', ''))
              
      allcombos = itertools.combinations(poswords, 2) # set this to the number of words you want in each combination
      for acombo in allcombos:
          row = []
          for anode in acombo:
              row.append(anode)
          row.append('Undirected')
          poswriter.writerow(row)

    if sentiment == 'Negative':
      negwords = []
      for aword in englishtext:
          if aword in negwordstoinclude:
              negwords.append(aword.replace(',', ''))
              
      allcombos = itertools.combinations(negwords, 2) # set this to the number of words you want in each combination
      for acombo in allcombos:
          row = []
          for anode in acombo:
              row.append(anode)
          row.append('Undirected')
          negwriter.writerow(row)

edgelist.close()
poslist.close()
neglist.close()

Writing Edge List
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
